# Project 2 
In this project we read data about user daily trajectories which content those data ("t_user_id", "transportation_mode", "collected_time", "latitude", "longitude") and from that data we calcualte those features for each user by each day also for each trasportation mode:
    ● Distance (e.g. Haversine)
    ● Speed (m/s)
    ● Acceleration (m/s2)
    ● Bearing (0 to 360 degrees)
in other py file we used the result to predict the transportation mode just from feature we have ( Distance, speed, Acceleration, Bearing).
For the preidction we will use two structres first one is Hierarchical Structre and second one is Flat Structre.

In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt
from geopy.distance import vincenty
from math import *
from datetime import datetime


In [8]:

# read the data from CSV file
all_data = pd.DataFrame(data=pd.read_csv('D:\Class\CSCI6515MLBigData\A2\geolife_raw.csv'))

# Put data in different virable to controll the size of it
data_frame = all_data
print("Original Data Rows count =", len(data_frame))

# Limatation o
limitation = 10


# Delete extra class from the data frame
def delete_class(df, class_todelete):
    for c in class_todelete:
        df = df[df.transportation_mode != c]
    return df


# Delete the Class run and motorcycle from the data frame
data_frame = delete_class(data_frame, ["motorcycle", "run"])
print("Delete two [\"motorcycle\", \"run\"] class  Data Rows count =", len(data_frame))

# Split the collected_time to col new_date and new_time
#  Delete the col collected_time
data_date = pd.to_datetime(data_frame.loc[:, "collected_time"])
data_frame = data_frame.drop("collected_time", axis=1)
date_val = [d.date() for d in data_date]
time_val = [d.time() for d in data_date]
data_frame['new_date'] = date_val  # The day
data_frame['new_time'] = time_val  # the time

# sort the data
data_frame = data_frame.sort_values(["t_user_id", "new_date", "new_time"])
# Change orgnizing of the columns
data_frame = data_frame[["t_user_id", "new_date", "transportation_mode", "new_time", "latitude", "longitude"]]
# Group the data by the
user = data_frame.groupby([data_frame.t_user_id, data_frame.new_date])
df = pd.DataFrame(data=np.array(user))



Original Data Rows count = 4485796
Delete two ["motorcycle", "run"] class  Data Rows count = 4483530


In [9]:

# Horizontal Bearing return value in degrees
def calcBearing(p1, p2):
    lat1 = p1[0]
    lon1 = p1[1]
    lat2 = p2[0]
    lon2 = p2[1]
    dLon = lon2 - lon1
    y = sin(dLon) * cos(lat2)
    x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dLon)
    bearing = degrees(atan2(y, x))
    return bearing


# Get time by second
def get_second(time):
    h, m, s = str(time).split(':')
    result = int(h) * 3600 + (int(m) * 60) + (int(s))
    if result == 0:
        result = 1
    return result


# cal_subtraj_features
# Calculate the Distance, Speed, Acceleration, Bearing for the sub_traj
def cal_subtraj_features(subtraj):
    last_acceleration = 0
    result = list()

    for i in range(1, len(subtraj)):
        # Start point
        p1_lat = subtraj[i - 1, 4]
        p1_log = subtraj[i - 1, 5]
        p1 = (p1_lat, p1_log)
        # End point
        p2_lat = subtraj[i, 4]
        p2_log = subtraj[i, 5]
        p2 = (p2_lat, p2_log)

        # Calculate the distance in KM
        distance = (vincenty(p1, p2).meters)

        # Calculate the diffrence between the times of to point
        t1 = str(subtraj[i - 1, 3])
        t2 = str(subtraj[i, 3])
        FMT = '%H:%M:%S'
        if datetime.strptime(t2, FMT) > datetime.strptime(t1, FMT):
            tdelta = datetime.strptime(t2, FMT) - datetime.strptime(t1, FMT)
        else:
            tdelta = datetime.strptime(t2, FMT) - datetime.strptime(t1, FMT)

        tdelta = datetime.strptime(str(tdelta), FMT).time()

        # Cal Speed by Km/h
        speed = distance / get_second(tdelta)

        # Cal Acceleration Km/h
        acceleration = abs(speed - last_acceleration) / pow(get_second(tdelta), 2)
        last_acceleration = acceleration

        # Calculate the Bearing Degrees
        bearing = calcBearing(p1, p2)
        # Shift the bearing
        TH = 180
        # if (bearing > TH): bearing -= 360;
        if (bearing < TH): bearing += 360;

        # convert to degree
        bearing *= pi / 180

        result.append(np.array([distance, speed, acceleration, bearing]))

    return pd.DataFrame(result)


# Feature data
# input sub_traj_data and calculate the feature
# distance, speed, acceleration, bearing

def get_feature_data(_class, sub_traj_data):
    row_list = list()
    describe_data = sub_traj_data.describe()
    describe_data.loc[len(describe_data)] = sub_traj_data.median(axis=0)

    row_list.append(_class)
    for i in range(len(describe_data.iloc[0])):
        col = describe_data.iloc[:, i]
        row_list.append(col[1])
        row_list.append(col[2])
        row_list.append(col[8])
        row_list.append(col[3])
        row_list.append(col[7])
    return (row_list)


# user_day_traj
# get user day traj and split to by transportation class
# return feature description for each sub_traj
def user_day_traj(user_day):
    _class = user_day.iloc[0, 2]  # first class
    sub_traj = list()  # list to save data for one class
    result = list()  # list to save final result
    for i in range(0, len(user_day)):
        # add data of same class to the sub_traj
        if (_class == user_day.iloc[i, 2]):
            sub_traj.append(user_day.iloc[i])
        else:
            # append the feature data for the class
            feature_class = cal_subtraj_features(np.array(sub_traj))
            if len(feature_class) >= limitation:
                feature_class = get_feature_data(_class, feature_class)
                result.append(feature_class)

            # change the class
            _class = user_day.iloc[i, 2]
            # empty the sub_traj
            sub_traj = list()
            sub_traj.append(user_day.iloc[i])
    feature_class = cal_subtraj_features(np.array(sub_traj))
    if len(feature_class) >= limitation:
        feature_class = get_feature_data(_class, feature_class)
        result.append(feature_class)
    return result


# Creat the DataFram for all the UsersTraj
# get Grouped data by (User_ID, Date)
# return Datafrom for the Data
def preprocess_data(all_groped_data):
    # convert grouped data to data frame
    df = pd.DataFrame(data=np.array(all_groped_data))
    result = list()
    # for each user and each day
    for i in range(0, len(df)):
        if i % 100 == 0:  # just to show the script still work
            print('-', sep=' ', end='', flush=True)
        # day data
        user_day = df.iloc[i, 1]
        if len(user_day) >= limitation:
            user_day_feature = user_day_traj(user_day)
            if len(user_day_feature) > 0:
                for udf in user_day_feature:
                    result.append(udf)
    print("Done")
    return pd.DataFrame(data=result, columns=[
        "trans_mode", "dis_mean", "dis_std", "dis_median", "dis_min", "dis_max"
        , "spe_mean", "spe_std", "spe_median", "spe_min", "spe_max"
        , "acc_mean", "acc_std", "acc_median", "acc_min", "acc_max"
        , "bea_mean", "bea_std", "bea_median", "bea_min", "bea_max"])



In [10]:

result = preprocess_data(df)

print("Shape of Data is =", result.shape)
result.to_csv(r'resultFinal.csv', index=False)


---------------------Done
Shape of Data is = (5832, 21)


In [15]:
import scipy.stats as sp
# this part to calculte t-test between the transportation mode for the {speed mean, acceleration mean, and distance mean} 
# to find if there any samlarity between those class.

# this method create datafram for the mean of those features {speed mean, acceleration mean, and distance mean} and
# reture the datafram
def describe_class(_all_data):
    bus = pd.DataFrame(_all_data[_all_data["trans_mode"] == 'bus'])
    car = pd.DataFrame(_all_data[_all_data["trans_mode"] == 'car'])
    walk = pd.DataFrame(_all_data[_all_data["trans_mode"] == 'walk'])
    taxi = pd.DataFrame(_all_data[_all_data["trans_mode"] == 'taxi'])
    subway = pd.DataFrame(_all_data[_all_data["trans_mode"] == 'subway'])
    train = pd.DataFrame(_all_data[_all_data["trans_mode"] == 'train'])

    bus_mean_spe_mean = bus["spe_mean"]
    bus_mean_acc_mean = bus["acc_mean"]
    bus_mean_dis_mean = bus["dis_mean"]

    car_mean_spe_mean = car["spe_mean"]
    car_mean_acc_mean = car["acc_mean"]
    car_mean_dis_mean = car["dis_mean"]

    walk_mean_spe_mean = walk["spe_mean"]
    walk_mean_acc_mean = walk["acc_mean"]
    walk_mean_dis_mean = walk["dis_mean"]

    taxi_mean_spe_mean = taxi["spe_mean"]
    taxi_mean_acc_mean = taxi["acc_mean"]
    taxi_mean_dis_mean = taxi["dis_mean"]

    subway_mean_spe_mean = subway["spe_mean"]
    subway_mean_acc_mean = subway["acc_mean"]
    subway_mean_dis_mean = subway["dis_mean"]

    train_mean_spe_mean = train["spe_mean"]
    train_mean_acc_mean = train["acc_mean"]
    train_mean_dis_mean = train["dis_mean"]

    result = list()
    result.append(["bus", bus_mean_spe_mean, bus_mean_acc_mean, bus_mean_dis_mean])
    result.append(["car", car_mean_spe_mean, car_mean_acc_mean, car_mean_dis_mean])
    result.append(["walk", walk_mean_spe_mean, walk_mean_acc_mean, walk_mean_dis_mean])
    result.append(["taxi", taxi_mean_spe_mean, taxi_mean_acc_mean, taxi_mean_dis_mean])
    result.append(["subway", subway_mean_spe_mean, subway_mean_acc_mean, subway_mean_dis_mean])
    result.append(["train", train_mean_spe_mean, train_mean_acc_mean, train_mean_dis_mean])
    return pd.DataFrame(data=result, columns=["class", "spe_mean", "acc_mean", "dis_mean"])


#  read the result from the file
data_frame = pd.DataFrame(data=pd.read_csv(r'resultFinal.csv'))

# pares of class to calculate the ttest for them and find the samilarity
pares = [["bus", "car"], ["bus", "walk"], ["bus", "taxi"], ["bus", "subway"], ["bus", "train"]
    , ["car", "walk"], ["car", "taxi"], ["car", "subway"], ["car", "train"]
    , ["walk", "taxi"], ["walk", "subway"], ["walk", "train"]
    , ["subway", "train"]]

# the features
data_measures = ["spe_mean", "acc_mean", "dis_mean"]

# create data fram for features
data_class_means = describe_class(data_frame)
# set the class as index to be easire for get the data
data_class_means.set_index("class", inplace=True)

# calculate the ttest
for j in data_measures:
    for i in range(len(pares)):
        p = pares[i]
        x = p[0]
        y = p[1]
        t_test_58_fd = 0.005
        data_class_x = data_class_means.loc[x][j]
        data_class_y = data_class_means.loc[y][j]
        statistic, p_val = sp.ttest_ind(data_class_x, data_class_y, equal_var=True)

        print("Measure ", j, ":")
        print('p_val', p_val, 'statistic', statistic)
        if p_val >= t_test_58_fd:
            print('the mean of ', x,
                  ' is identical to mean of Speed', y)
        else:
            print('the mean of ', x,
                  'is not identical to mean of model', y)
    print("______________________________________________")


5832
Measure  spe_mean :
p_val 4.76642940887e-108 statistic -23.5249863037
the mean of  bus is not identical to mean of model car
Measure  spe_mean :
p_val 0.0 statistic 65.2687973572
the mean of  bus is not identical to mean of model walk
Measure  spe_mean :
p_val 7.01341063315e-84 statistic -20.4134876108
the mean of  bus is not identical to mean of model taxi
Measure  spe_mean :
p_val 2.29375450136e-118 statistic -24.7861827683
the mean of  bus is not identical to mean of model subway
Measure  spe_mean :
p_val 0.0 statistic -62.7416438141
the mean of  bus is not identical to mean of model train
Measure  spe_mean :
p_val 0.0 statistic 76.0855639473
the mean of  car is not identical to mean of model walk
Measure  spe_mean :
p_val 0.0903408975559 statistic 1.69540991591
the mean of  car  is identical to mean of Speed taxi
Measure  spe_mean :
p_val 0.000773304055671 statistic -3.37260086855
the mean of  car is not identical to mean of model subway
Measure  spe_mean :
p_val 2.63370779644